# 1.experiment setting

In [1]:
## main.py

import os
import argparse
import logging
import random
import numpy as np
import torch
from torch.utils.data import DataLoader, random_split

from build_vocab import build_dictionary
from dataset import CustomTextDataset, collate_fn
from model import RCNN
from trainer import train, evaluate
from utils import read_file

/home/dsail/hyolim/hyolim_py38/lib/python3.8/site-packages/tqdm/auto.py:22: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm
[nltk_data] Downloading package punkt to /home/dsail/nltk_data...
[nltk_data]   Package punkt is already up-to-date!


## 1.1 argparse
https://docs.python.org/3/library/argparse.html?highlight=argparse#type

* 1. argumentparser에 원하는 description을 입력하여 parser 객체를 생성한다
* 2. add_argument()로 argument는 원하는 만큼 추가가 가능하다
* 3. parse_args()로 주어진 인자를 파싱한다


In [2]:
## main.py

parser = argparse.ArgumentParser()
parser.add_argument('--seed', type=int, default=42)
parser.add_argument('--test_set', action='store_true', default=False) # 'store_true'=special cases of 'store_const' used for storing the values

# data
parser.add_argument("--train_file_path", type=str, default="./data/train.csv")
parser.add_argument("--test_file_path", type=str, default="./data/test.csv")
parser.add_argument("--model_save_path", type=str, default="./model_saved")
parser.add_argument("--num_val_data", type=int, default=10000)
parser.add_argument("--max_len", type=int, default=64)
parser.add_argument("--batch_size", type=int, default=64)

# model
parser.add_argument("--vocab_size", type=int, default=8000)
parser.add_argument("--embedding_dim", type=int, default=300)
parser.add_argument("--hidden_size", type=int, default=512)
parser.add_argument("--hidden_size_linear", type=int, default=512)
parser.add_argument("--class_num", type=int, default=4)
parser.add_argument("--dropout", type=float, default=0.0)

# training
parser.add_argument("--epochs", type=int, default=10)
parser.add_argument("--lr", type=float, default=3e-4)
args = parser.parse_args(args=[])

In [3]:
## main.py

def set_seed(args): # set_seed(args) 시행 위해 함수 정의
    random.seed(args.seed)
    np.random.seed(args.seed)
    torch.manual_seed(args.seed)
    if args.n_gpu > 0:
        torch.cuda.manual_seed_all(args.seed)

args.device = torch.device("cuda:1" if torch.cuda.is_available() else "cpu")
args.n_gpu = torch.cuda.device_count()
set_seed(args)

args

Namespace(batch_size=64, class_num=4, device=device(type='cuda', index=1), dropout=0.0, embedding_dim=300, epochs=10, hidden_size=512, hidden_size_linear=512, lr=0.0003, max_len=64, model_save_path='./model_saved', n_gpu=2, num_val_data=10000, seed=42, test_file_path='./data/test.csv', test_set=False, train_file_path='./data/train.csv', vocab_size=8000)

# 2. data load
dataset
* train 110,000
* valid 10,000
* test 7,600

classes: Original classes are 1, 2, 3, 4 each, but changed them into 0, 1, 2, 3.
(바꾸어 주는 이유: 0부터 시작을 안하면 딥러닝이 인식을 못하고 에러 뜬다)
* labels = [label-1 for label in list(data['class'].values)]  # label : 1-4  -> label : 0-3

    * 0: World
    * 1: Sports
    * 2: Business
    * 3: Sci/Tech

In [4]:
## main.py

train_texts, train_labels = read_file(args.train_file_path)

## 2.1 word2idx, 워드임베딩에 필요한 단어 사전 생성 
### collection 모듈의 counter 클래스 사용

https://docs.python.org/ko/3/library/collections.html#collections.Counter

* 단어 개수를 세는 도구로 주로 사용
* update([iterable-or-mapping])
* most_common([n]): n개의 가장 많은 요소와 그 개수를 가장 많은 것부터 가장 적은 것 순으로 리스트 반환

In [5]:
texts=train_texts 
# word2idx를 만들때는 train 데이터만 사용한다, test를 사용하는 것은 정답을 미리 알려주는 것이기에 의미가 없기 때문이다

## build_vocab.py

In [6]:
def build_dictionary(texts, vocab_size):
    counter = Counter()                  
    SPECIAL_TOKENS = ['<PAD>', '<UNK>']

    for word in texts:
        counter.update(word)

    words = [word for word, count in counter.most_common(vocab_size - len(SPECIAL_TOKENS))]
    words = SPECIAL_TOKENS + words
    word2idx = {word: idx for idx, word in enumerate(words)}

    return word2idx

In [7]:
## build_vocab.py

from collections import Counter

counter = Counter()  # 빈 새로운 counter를 만듦
counter

Counter()

### PAD와 UNK

http://tensorboy.com/padding-for-nlp
* pad: 문장 길이를 맞추기 위한
* unk: 없는 글자를 처리하기 위한

* PAD
    * NLP에서 입력인 문장은 길이가 다양함, 여러 문장들을 matrix 형태로 모델에 입력하기 위해서 길이를 같게 만들어줘야 하는데 이 행위를 padding이라 하며 이때 사용되는 특수 token이 [PAD] token
    *  특수 token [PAD]를 짧은 문장의 뒤에 추가해서 길이가 같게 만들어 주는 원리

In [8]:
SPECIAL_TOKENS = ['<PAD>', '<UNK>']
SPECIAL_TOKENS 

['<PAD>', '<UNK>']

In [9]:
for word in texts:
    counter.update(word)
    # break # break 사용해서 하나만 확인

In [10]:
counter # counter.update(word) 로 인해 {문자 : 개수}의 딕셔너리 형태로 개수가 세어짐

Counter({'wall': 1424,
         'st': 1424,
         'bears': 401,
         'claw': 17,
         'back': 4491,
         'into': 6677,
         'the': 205460,
         'black': 788,
         '\\': 87592,
         '(': 41106,
         'reuters': 19340,
         ')': 40787,
         'short': 897,
         'sellers': 47,
         ',': 165685,
         'street': 1618,
         "'s": 22500,
         'dwindling': 32,
         'band': 212,
         'of': 98652,
         'ultra': 81,
         'cynics': 5,
         'are': 9873,
         'seeing': 136,
         'green': 843,
         'again': 1799,
         'carlyle': 16,
         'looks': 600,
         'toward': 760,
         'commercial': 506,
         'aerospace': 128,
         'private': 725,
         'investment': 828,
         'firm': 1785,
         'group': 4714,
         'which': 4606,
         'has': 19085,
         'a': 113353,
         'reputation': 117,
         'for': 50671,
         'making': 1262,
         'well': 1262,
         't

In [11]:
vocab_size=args.vocab_size # 8000
len(SPECIAL_TOKENS)

2

In [12]:
len(counter.most_common(vocab_size - len(SPECIAL_TOKENS))) 
# 7998, 나중에 SPECIAL TOKENS 두 자리를 채우기 위해서 8000개가 아닌 7998개만 뽑아낸다
counter.most_common(vocab_size - len(SPECIAL_TOKENS)) # 8000-2개의 요소들을 순서 많은 것부터 차례대로 나열하라

[('the', 205460),
 (',', 165685),
 ('to', 120720),
 ('a', 113353),
 ('of', 98652),
 ('in', 96420),
 ('\\', 87592),
 ('and', 69677),
 ('on', 57658),
 ('for', 50671),
 ('39', 44506),
 ('(', 41106),
 (')', 40787),
 ('s', 39484),
 ('that', 28169),
 ('with', 26812),
 ('as', 25376),
 ('at', 25067),
 ("'s", 22500),
 ('is', 22269),
 ('its', 22123),
 ('new', 21423),
 ('by', 20939),
 ('it', 20536),
 ('said', 20267),
 ('reuters', 19340),
 ('has', 19085),
 ('from', 17828),
 ('an', 17000),
 ('ap', 16276),
 ('his', 14942),
 ('will', 14610),
 ('after', 14551),
 ('was', 13836),
 ('gt', 13231),
 ('us', 13229),
 ('lt', 13182),
 ('be', 11834),
 ('over', 11356),
 ('have', 11257),
 ('up', 10740),
 ('their', 10530),
 ('two', 10224),
 ('are', 9873),
 ('first', 9799),
 ('year', 9772),
 ('quot', 9596),
 ('but', 9185),
 ('more', 9144),
 ('he', 8948),
 ('world', 8626),
 ('u', 8440),
 ('this', 8247),
 ('1', 8226),
 ('one', 8105),
 ('company', 7656),
 ('monday', 7614),
 ('oil', 7563),
 ('out', 7554),
 ('wednesday'

In [13]:
words = [word for word, count in counter.most_common(vocab_size - len(SPECIAL_TOKENS))]
words # 7998개의 단어를 가장 많이 나온 순서대로 나열함

['the',
 ',',
 'to',
 'a',
 'of',
 'in',
 '\\',
 'and',
 'on',
 'for',
 '39',
 '(',
 ')',
 's',
 'that',
 'with',
 'as',
 'at',
 "'s",
 'is',
 'its',
 'new',
 'by',
 'it',
 'said',
 'reuters',
 'has',
 'from',
 'an',
 'ap',
 'his',
 'will',
 'after',
 'was',
 'gt',
 'us',
 'lt',
 'be',
 'over',
 'have',
 'up',
 'their',
 'two',
 'are',
 'first',
 'year',
 'quot',
 'but',
 'more',
 'he',
 'world',
 'u',
 'this',
 '1',
 'one',
 'company',
 'monday',
 'oil',
 'out',
 'wednesday',
 'tuesday',
 'thursday',
 'not',
 '2',
 'against',
 'friday',
 'inc',
 'than',
 'into',
 'last',
 'they',
 'about',
 'iraq',
 'b',
 'york',
 'yesterday',
 'who',
 'three',
 'president',
 'microsoft',
 'no',
 'were',
 'game',
 'million',
 '?',
 'week',
 'been',
 'time',
 'says',
 'had',
 'corp',
 'united',
 'when',
 'com',
 'stocks',
 'sunday',
 'could',
 'prices',
 'would',
 'government',
 '3',
 'security',
 'years',
 'group',
 'today',
 'n',
 'people',
 'off',
 'which',
 'may',
 'second',
 'afp',
 'percent',
 'b

In [14]:
words = SPECIAL_TOKENS + words
words # SPECIAL_TOKENS를 추가해서 8000개로 만들어줌

['<PAD>',
 '<UNK>',
 'the',
 ',',
 'to',
 'a',
 'of',
 'in',
 '\\',
 'and',
 'on',
 'for',
 '39',
 '(',
 ')',
 's',
 'that',
 'with',
 'as',
 'at',
 "'s",
 'is',
 'its',
 'new',
 'by',
 'it',
 'said',
 'reuters',
 'has',
 'from',
 'an',
 'ap',
 'his',
 'will',
 'after',
 'was',
 'gt',
 'us',
 'lt',
 'be',
 'over',
 'have',
 'up',
 'their',
 'two',
 'are',
 'first',
 'year',
 'quot',
 'but',
 'more',
 'he',
 'world',
 'u',
 'this',
 '1',
 'one',
 'company',
 'monday',
 'oil',
 'out',
 'wednesday',
 'tuesday',
 'thursday',
 'not',
 '2',
 'against',
 'friday',
 'inc',
 'than',
 'into',
 'last',
 'they',
 'about',
 'iraq',
 'b',
 'york',
 'yesterday',
 'who',
 'three',
 'president',
 'microsoft',
 'no',
 'were',
 'game',
 'million',
 '?',
 'week',
 'been',
 'time',
 'says',
 'had',
 'corp',
 'united',
 'when',
 'com',
 'stocks',
 'sunday',
 'could',
 'prices',
 'would',
 'government',
 '3',
 'security',
 'years',
 'group',
 'today',
 'n',
 'people',
 'off',
 'which',
 'may',
 'second',
 'a

In [15]:
word2idx = {word: idx for idx, word in enumerate(words)} 
# enumerate()함수: 인덱스와 원소를 동시에 접근하면서 for문을 돌릴 수 있도록 함
word2idx # 워드임베딩에 필요한 단어사전

# 앞에서 counter.most_common를 활용해서 가장 많이 나오는 단어가 앞의 인덱스를 가질 수 있도록 배정하였음

{'<PAD>': 0,
 '<UNK>': 1,
 'the': 2,
 ',': 3,
 'to': 4,
 'a': 5,
 'of': 6,
 'in': 7,
 '\\': 8,
 'and': 9,
 'on': 10,
 'for': 11,
 '39': 12,
 '(': 13,
 ')': 14,
 's': 15,
 'that': 16,
 'with': 17,
 'as': 18,
 'at': 19,
 "'s": 20,
 'is': 21,
 'its': 22,
 'new': 23,
 'by': 24,
 'it': 25,
 'said': 26,
 'reuters': 27,
 'has': 28,
 'from': 29,
 'an': 30,
 'ap': 31,
 'his': 32,
 'will': 33,
 'after': 34,
 'was': 35,
 'gt': 36,
 'us': 37,
 'lt': 38,
 'be': 39,
 'over': 40,
 'have': 41,
 'up': 42,
 'their': 43,
 'two': 44,
 'are': 45,
 'first': 46,
 'year': 47,
 'quot': 48,
 'but': 49,
 'more': 50,
 'he': 51,
 'world': 52,
 'u': 53,
 'this': 54,
 '1': 55,
 'one': 56,
 'company': 57,
 'monday': 58,
 'oil': 59,
 'out': 60,
 'wednesday': 61,
 'tuesday': 62,
 'thursday': 63,
 'not': 64,
 '2': 65,
 'against': 66,
 'friday': 67,
 'inc': 68,
 'than': 69,
 'into': 70,
 'last': 71,
 'they': 72,
 'about': 73,
 'iraq': 74,
 'b': 75,
 'york': 76,
 'yesterday': 77,
 'who': 78,
 'three': 79,
 'president': 80

In [16]:
#word2idx = build_dictionary(train_texts, vocab_size=args.vocab_size)
word2idx 

{'<PAD>': 0,
 '<UNK>': 1,
 'the': 2,
 ',': 3,
 'to': 4,
 'a': 5,
 'of': 6,
 'in': 7,
 '\\': 8,
 'and': 9,
 'on': 10,
 'for': 11,
 '39': 12,
 '(': 13,
 ')': 14,
 's': 15,
 'that': 16,
 'with': 17,
 'as': 18,
 'at': 19,
 "'s": 20,
 'is': 21,
 'its': 22,
 'new': 23,
 'by': 24,
 'it': 25,
 'said': 26,
 'reuters': 27,
 'has': 28,
 'from': 29,
 'an': 30,
 'ap': 31,
 'his': 32,
 'will': 33,
 'after': 34,
 'was': 35,
 'gt': 36,
 'us': 37,
 'lt': 38,
 'be': 39,
 'over': 40,
 'have': 41,
 'up': 42,
 'their': 43,
 'two': 44,
 'are': 45,
 'first': 46,
 'year': 47,
 'quot': 48,
 'but': 49,
 'more': 50,
 'he': 51,
 'world': 52,
 'u': 53,
 'this': 54,
 '1': 55,
 'one': 56,
 'company': 57,
 'monday': 58,
 'oil': 59,
 'out': 60,
 'wednesday': 61,
 'tuesday': 62,
 'thursday': 63,
 'not': 64,
 '2': 65,
 'against': 66,
 'friday': 67,
 'inc': 68,
 'than': 69,
 'into': 70,
 'last': 71,
 'they': 72,
 'about': 73,
 'iraq': 74,
 'b': 75,
 'york': 76,
 'yesterday': 77,
 'who': 78,
 'three': 79,
 'president': 80

### 데이터배치와 반복자 생성하기
https://tutorials.pytorch.kr/beginner/text_sentiment_ngrams_tutorial.html

1. collate_fn()
    * 모델로 보내기 전, DataLoader에서 생성된 샘플 배치로 동작함
    * collate_fn의 입력은 DataLoader에 배치 크기가 있는 배치 데이터임
    * 미리 선언된 데이터 처리 파이프라인에 따라 처리

In [17]:
full_dataset = CustomTextDataset(train_texts, train_labels, word2idx)

num_train_data = len(full_dataset) - args.num_val_data

train_dataset, val_dataset = random_split(full_dataset, [num_train_data, args.num_val_data])

train_dataloader = DataLoader(dataset=train_dataset,
                              collate_fn=lambda x: collate_fn(x, args),
                              batch_size=args.batch_size,
                              shuffle=True)

valid_dataloader = DataLoader(dataset=val_dataset,
                              collate_fn=lambda x: collate_fn(x, args),
                              batch_size=args.batch_size,
                              shuffle=True)

In [18]:
print(len(train_texts[0]))
print(train_texts[0])

print(len(train_texts[0][0]))
print(train_texts[0][0])

print(len(train_texts[0][1]))
print(train_texts[0][1])

# '\\': 백슬래시

30
['wall', 'st', 'bears', 'claw', 'back', 'into', 'the', 'black', '\\', '(', 'reuters', '\\', ')', 'reuters', 'short', 'sellers', ',', 'wall', 'street', "'s", 'dwindling', 'band', 'of', 'ultra', 'cynics', ',', 'are', 'seeing', 'green', 'again']
4
wall
2
st


In [19]:
test_texts, test_labels = read_file(args.test_file_path)
test_dataset = CustomTextDataset(test_texts, test_labels, word2idx)
test_dataloader = DataLoader(dataset=test_dataset,
                             collate_fn=lambda x: collate_fn(x, args),
                             batch_size=args.batch_size,
                             shuffle=True)

## dataset.py

In [20]:
import torch
from torch.utils.data import Dataset

# 여기서부터 쭉 새롭게 함
### custom dataset(커스텀 데이터셋)

https://wikidocs.net/57165

* 데이터셋을 상속받아 직접 커스텀 데이터셋을 만드는 것으로
* torch.utils.data.Dataset은 파이토치에서 데이터셋을 제공하는 추상 클래스

#### 커스텀 데이터셋 기본 뼈대
* class CustomDataset(torch.utils.data.Dataset): 
  * def __init__(self):           # 데이터셋의 전처리를 해주는 부분

  * def __len__(self):            # 데이터셋의 길이. 즉, 총 샘플의 수를 적어주는 부분
                                # 데이터셋의 크기를 리턴할 len
  * def __getitem__(self, idx):  # 데이터셋에서 특정 1개의 샘플을 가져오는 함수
                               # dataset[i]을 했을 때 i번째 샘플을 가져오도록 하는 인덱싱을 위한 get_item

In [21]:
class CustomTextDataset(Dataset): 
    def __init__(self, texts, labels, dictionary):
        # Unknown Token is index 1 (<UNK>)
        self.x = [[dictionary.get(token, 1) for token in token_list] for token_list in texts]
        self.y = labels

    def __len__(self):
        """Return the data length"""
        return len(self.x)

    def __getitem__(self, idx):
        """Return one item on the index"""
        return self.x[idx], self.y[idx]

### __init__

In [22]:
# CustomTextDataset(test_texts, test_labels, word2idx)이기에 def의 dictionary를 word2idx로 바꾸어줌
x = [[word2idx.get(token, 1) for token in token_list] for token_list in texts]
x # (0~8000사이의 숫자들)

[[458,
  459,
  1654,
  1,
  115,
  70,
  2,
  860,
  8,
  13,
  27,
  8,
  14,
  27,
  766,
  1,
  3,
  458,
  397,
  20,
  1,
  2901,
  6,
  5780,
  1,
  3,
  45,
  4048,
  807,
  349],
 [1,
  1113,
  891,
  1319,
  4245,
  8,
  13,
  27,
  8,
  14,
  27,
  935,
  826,
  352,
  1,
  105,
  3,
  110,
  28,
  5,
  4511,
  11,
  525,
  526,
  1,
  9,
  1,
  1534,
  2194,
  7,
  2,
  548,
  253,
  3,
  28,
  3939,
  2313,
  22,
  6527,
  10,
  226,
  382,
  6,
  2,
  136],
 [59,
  9,
  380,
  4573,
  96,
  124,
  778,
  8,
  13,
  27,
  8,
  14,
  27,
  2431,
  480,
  99,
  1920,
  1300,
  73,
  2,
  380,
  9,
  2,
  778,
  11,
  294,
  45,
  201,
  4,
  5781,
  40,
  2,
  306,
  136,
  118,
  87,
  243,
  2,
  6177,
  6,
  2,
  1191,
  1],
 [74,
  7351,
  59,
  1858,
  29,
  907,
  569,
  2902,
  8,
  13,
  27,
  8,
  14,
  27,
  873,
  41,
  4930,
  59,
  3598,
  7843,
  29,
  2,
  907,
  2902,
  7,
  569,
  74,
  34,
  1473,
  739,
  5,
  794,
  2579,
  98,
  530,
  3065,
  3,
  30,
 

### __getitem__

In [23]:
len(x[9]) # 31
len(x[19]) # 29
# idx 대신 숫자를 넣어서 특정 1개의 샘플 추출 가능

29


## collate_fn

https://velog.io/@jiyoung/torch-collatefn%EC%97%90-arguments-%EC%B6%94%EA%B0%80%ED%95%98%EA%B8%B0

* collate 과정을 거쳐 생성된 batched data를 dataloader에 feeding 하는 것은 위와 같이 Collator 인스턴스를 생성 후 data_loader에 'collate_fn=생성된 인스턴스'로 넘겨주면 된다.

* dataset이 고정된 길이가 아닐 경우, batchsize를 2 이상으로 dataloader를 호출하면 dataloader에서 batch로 바로 못묶이고 에러가 난다. 따라서 텍스트데이터와 같이 variable length data를 다루고 batchsize를 2 이상으로 주고자 하는 경우에는 collate_fn 함수를 직접 작성해 넘겨주어야 한다.

In [24]:
def collate_fn(data, args, pad_idx=0):
    """Padding"""
    texts, labels = zip(*data) # zip(*args인수): * 사용시 col끼리 서로 엮어짐
    texts = [s + [pad_idx] * (args.max_len - len(s)) if len(s) < args.max_len else s[:args.max_len] for s in texts]
    return torch.LongTensor(texts), torch.LongTensor(labels)

#### torch.LongTensor()
https://wikidocs.net/52846

* 텐서의 자료형(타입 캐스팅)
    * 데이터형별로 정의되어있으며 64비트의 부호있는 정수의 경우 torch.LongTensor 사용

In [25]:
pad_idx=0
texts = [s + [pad_idx] * (args.max_len - len(s)) if len(s) < args.max_len else s[:args.max_len] for s in texts]
texts
# padding을 위한 코드,
# args.max_len(64)보다 작으면 해당 텍스트 + 빈자리만큼*0(0곱해서 채움)
# 크면 [:64]로 64개만 나오도록 조정

[['wall',
  'st',
  'bears',
  'claw',
  'back',
  'into',
  'the',
  'black',
  '\\',
  '(',
  'reuters',
  '\\',
  ')',
  'reuters',
  'short',
  'sellers',
  ',',
  'wall',
  'street',
  "'s",
  'dwindling',
  'band',
  'of',
  'ultra',
  'cynics',
  ',',
  'are',
  'seeing',
  'green',
  'again',
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0],
 ['carlyle',
  'looks',
  'toward',
  'commercial',
  'aerospace',
  '\\',
  '(',
  'reuters',
  '\\',
  ')',
  'reuters',
  'private',
  'investment',
  'firm',
  'carlyle',
  'group',
  ',',
  'which',
  'has',
  'a',
  'reputation',
  'for',
  'making',
  'well',
  'timed',
  'and',
  'occasionally',
  'controversial',
  'plays',
  'in',
  'the',
  'defense',
  'industry',
  ',',
  'has',
  'quietly',
  'placed',
  'its',
  'bets',
  'on',
  'another',
  'part',
  'of',
  'the',
  'market',
  0,
  0,
  0,
  0,
  0,
  0

In [26]:
args.max_len

64

In [27]:
print(len(train_labels)) # 120000
print(len(train_texts))

print(len(test_labels)) # 7600
print(len(test_texts))
len(test_dataset)

120000
120000
7600
7600


7600

trainers.py의 
* 'for step, (x, y) in enumerate(train_dataloader):
*            x, y = x.to(args.device), y.to(args.device)'

* 부분을 통해 x와 y의 인풋       

In [28]:
test_dataset.x # 1~8000 사이의 값
# 사전에 없는 단어는 <UNK> 1, 0은 존재하지 않음(<PAD>안해주어서 그냥 원래 데이터니까)

[[899,
  11,
  148,
  107,
  1513,
  34,
  188,
  1815,
  4120,
  434,
  19,
  6600,
  1,
  254,
  72,
  45,
  1,
  124,
  34,
  188,
  17,
  7697,
  2514,
  352,
  211,
  1],
 [2,
  494,
  21,
  10,
  112,
  935,
  120,
  1112,
  456,
  1788,
  11,
  744,
  1,
  8,
  13,
  192,
  95,
  8,
  14,
  192,
  95,
  756,
  3,
  594,
  5,
  112,
  120,
  6,
  1,
  4024,
  11,
  2,
  419,
  142,
  85,
  6113,
  1350,
  1091,
  3,
  5,
  3601,
  11,
  3377,
  6084,
  1,
  192,
  811,
  3,
  28,
  2506,
  156,
  2,
  46,
  456,
  1788,
  11,
  22,
  4411,
  1182],
 [1,
  57,
  444,
  4409,
  4,
  536,
  1,
  8,
  13,
  31,
  8,
  14,
  31,
  5,
  57,
  6730,
  24,
  5,
  1,
  5110,
  19,
  2,
  635,
  6,
  4212,
  257,
  5,
  4409,
  4,
  1433,
  5,
  7203,
  6,
  3443,
  620,
  1,
  3,
  110,
  45,
  766,
  6909,
  6,
  1,
  1,
  3,
  2,
  1085,
  4444,
  6,
  1],
 [1,
  653,
  2125,
  753,
  1,
  8,
  13,
  31,
  8,
  14,
  31,
  25,
  20,
  3539,
  4155,
  94,
  1159,
  1,
  1875,
  32,
  228

In [29]:
test_dataset.x[100]

[336,
 648,
 11,
 206,
 3,
 1,
 30,
 727,
 503,
 215,
 573,
 32,
 255,
 4,
 32,
 195,
 12,
 15,
 46,
 707,
 3357,
 336,
 2800,
 723,
 10,
 62,
 3,
 241,
 7,
 2,
 666,
 1178,
 30,
 1393,
 6,
 3854,
 12,
 15,
 890,
 842,
 1405,
 2,
 46,
 707,
 7067,
 29,
 2,
 93,
 1557,
 5620]

In [30]:
test_dataset.y # 0,1,2,3

[2,
 3,
 3,
 3,
 3,
 3,
 3,
 3,
 3,
 3,
 3,
 3,
 3,
 3,
 3,
 3,
 3,
 3,
 3,
 3,
 3,
 3,
 3,
 3,
 3,
 3,
 1,
 1,
 1,
 1,
 1,
 1,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 1,
 2,
 0,
 3,
 1,
 1,
 0,
 3,
 0,
 1,
 0,
 1,
 0,
 3,
 2,
 3,
 0,
 0,
 2,
 2,
 1,
 1,
 1,
 3,
 0,
 3,
 0,
 0,
 1,
 0,
 3,
 3,
 3,
 0,
 3,
 1,
 0,
 1,
 0,
 0,
 0,
 1,
 2,
 2,
 0,
 0,
 2,
 0,
 0,
 3,
 0,
 2,
 3,
 2,
 1,
 1,
 1,
 2,
 0,
 2,
 1,
 2,
 3,
 2,
 0,
 1,
 0,
 1,
 0,
 3,
 2,
 3,
 3,
 3,
 3,
 1,
 3,
 3,
 2,
 1,
 0,
 1,
 3,
 0,
 3,
 2,
 1,
 1,
 1,
 0,
 0,
 0,
 1,
 0,
 1,
 1,
 0,
 1,
 1,
 0,
 1,
 0,
 1,
 0,
 0,
 0,
 1,
 2,
 1,
 1,
 1,
 0,
 1,
 0,
 0,
 0,
 2,
 1,
 1,
 0,
 1,
 2,
 3,
 0,
 0,
 2,
 2,
 2,
 1,
 0,
 3,
 3,
 2,
 3,
 0,
 0,
 3,
 1,
 3,
 1,
 2,
 1,
 1,
 3,
 2,
 0,
 3,
 0,
 1,
 3,
 3,
 0,
 0,
 0,
 2,
 2,
 0,
 1,
 2,
 1,
 3,
 3,
 3,
 0,
 1,
 1,
 1,
 3,
 1,
 3,
 1,
 0,
 1,
 1,
 1,
 3,
 2,
 2,
 3,
 1,
 1,
 0,
 2,
 1,
 2,
 2,
 0,
 1,
 2,
 0,
 1,
 1,
 2,
 3,
 2,
 1,
 2,
 1,
 0,
 2,
 3,
 1,
 1,
 3,
 2,
 2,
 3,
 3,
 3,
 0,


In [31]:
len(test_dataset.y), len(test_dataset.x)

(7600, 7600)

# 3. model
## trainers.py

3. Methodology
- 인풋: D(document) = sequence of words w1, w2 . . . wn
- 아웃풋: class elements 포함*(각 4개 데이터셋의 라벨들)*

## 3.1 x, y(input) 확인하기

In [32]:
# args.device를 안해주면?
for step, (x, y) in enumerate(train_dataloader):
    x, y = x.to(args.device), y.to(args.device) 
    break # break 사용해서 하나만 확인, 가장 첫번째거, step 확인시 0 출력
    # x, y #: 이 코드로 시행시 가장 마지막 배치 출력

In [33]:
x # 남는 공간은 0으로 채움

tensor([[2075,  955,  519,  ...,    0,    0,    0],
        [3750,   19,   71,  ...,    0,    0,    0],
        [ 725,  845,   17,  ...,    0,    0,    0],
        ...,
        [3442, 2886,   82,  ...,    0,    0,    0],
        [   7, 3130,    1,  ...,    0,    0,    0],
        [ 160,  443, 2825,  ...,    0,    0,    0]], device='cuda:1')

In [34]:
x.shape # torch.Size([batch_size, max_len])

torch.Size([64, 64])

#### PAD: 0, UNK: 1
* pad: 문장 길이를 맞추기 위한
* unk: 없는 글자를 처리하기 위한

In [35]:
len(x[0]) # 64, 길이를 모두 64로 통일, 부족한 부분은 pad토큰을 통해 0으로 채워서
x[0]

tensor([2075,  955,  519,    4, 1221,    7,  655,  211, 2075,   67, 1108,    5,
         589,    7,  655,    4,  511,    2,  948,  519,    6,   79,  160, 1480,
          92,  894,    7,    2,  556,    1,   20, 5897,    3,  640,   16,    5,
        2757,  519,   21,  684,    0,    0,    0,    0,    0,    0,    0,    0,
           0,    0,    0,    0,    0,    0,    0,    0,    0,    0,    0,    0,
           0,    0,    0,    0], device='cuda:1')

In [36]:
len(x[1]) # 64
x[1]

tensor([3750,   19,   71,   11,    1,   78,    1, 2207,    1,  261,    1,    3,
         692,    8,   13,   27,    8,   14, 1027,    6,    1,   78, 3042, 6112,
           2, 2207,   66,   53,   15,  484,  387,   34,    2,  797,  332,    3,
        1957,    3,  439,   10,    2,   93,  177,   83,  378,   29,   30,  751,
        1601,   97,   34,  427,   79,  104,   18, 2139,    6,  259,    0,    0,
           0,    0,    0,    0], device='cuda:1')

In [37]:
y # 0, 1, 2, 3

tensor([2, 0, 3, 0, 1, 2, 0, 0, 1, 3, 1, 2, 1, 3, 2, 0, 1, 2, 3, 2, 3, 0, 2, 0,
        1, 2, 2, 3, 2, 2, 3, 0, 2, 3, 3, 0, 2, 2, 3, 0, 0, 1, 1, 3, 1, 3, 1, 2,
        1, 0, 3, 0, 2, 1, 0, 1, 3, 3, 0, 0, 0, 3, 3, 1], device='cuda:1')

In [38]:
y.shape

torch.Size([64])

## 3.2 RCNN model
(훈련 전 모델 먼저 만들기)

* class RCNN(nn.Module):
    
    * Recurrent Convolutional Neural Networks for Text Classification (2015)
    
    * def __init__(self, vocab_size, embedding_dim, hidden_size, hidden_size_linear, class_num, dropout):

In [39]:
vocab_size=args.vocab_size
embedding_dim=args.embedding_dim
hidden_size=args.hidden_size
hidden_size_linear=args.hidden_size_linear
class_num=args.class_num
dropout=args.dropout

### nn.DataParallel

https://tutorials.pytorch.kr/beginner/blitz/data_parallel_tutorial.html

* (선택사항) 데이터 병렬 처리
    * 여러 GPU를 사용하는 경우가 많은데, pytorch에서는 기본적으로 하나의 GPU만 사용, 이때 DataParallel을 이용해 모델을 병렬로 시행해, 다수의 GPU에서 쉽게 작업 실행 가능

In [40]:
model = RCNN(vocab_size=args.vocab_size,
             embedding_dim=args.embedding_dim,
             hidden_size=args.hidden_size,
             hidden_size_linear=args.hidden_size_linear,
             class_num=args.class_num,
             dropout=args.dropout).to(args.device) 
            # .to(device): 모든 텐서를 GPU로 복사

# if args.n_gpu > 1:
#        model = torch.nn.DataParallel(model, dim=0)

model

RCNN(
  (embedding): Embedding(8000, 300, padding_idx=0)
  (lstm): LSTM(300, 512, batch_first=True, bidirectional=True)
  (W): Linear(in_features=1324, out_features=512, bias=True)
  (tanh): Tanh()
  (fc): Linear(in_features=512, out_features=4, bias=True)
)

#### torch.optim
* 최적화: 신경망을 훈련시키는 동안 네트워크의 가중치나 매개변수를 조정하고자 오차를 역전파하는 과정
- opt=optim.Adam(moedel.parameters(),lr=lr)
* 아래 두 단계의 경우 train 함수 안에 작성
    - opt.step()
    - opt.zero_grad()

In [41]:
optimizer = torch.optim.Adam(model.parameters(), lr=args.lr)
optimizer
# train(model, optimizer, train_dataloader, valid_dataloader, args) 
# 해당 코드 시행시 모델 훈련 시작

Adam (
Parameter Group 0
    amsgrad: False
    betas: (0.9, 0.999)
    eps: 1e-08
    lr: 0.0003
    weight_decay: 0
)

## RCNN nn.module

In [42]:
import torch
import torch.nn as nn
import torch.nn.functional as F

In [43]:
class RCNN(nn.Module):
    """
    Recurrent Convolutional Neural Networks for Text Classification (2015)
    """
    def __init__(self, vocab_size, embedding_dim, hidden_size, hidden_size_linear, class_num, dropout):
        super(RCNN, self).__init__()
        self.embedding = nn.Embedding(vocab_size, embedding_dim, padding_idx=0)
        self.lstm = nn.LSTM(embedding_dim, hidden_size, batch_first=True, bidirectional=True, dropout=dropout)
        self.W = nn.Linear(embedding_dim + 2*hidden_size, hidden_size_linear)
        self.tanh = nn.Tanh()
        self.fc = nn.Linear(hidden_size_linear, class_num)

    def forward(self, x):
        # x = |bs, seq_len|
        x_emb = self.embedding(x)
        # x_emb = |bs, seq_len, embedding_dim|
        output, _ = self.lstm(x_emb)
        # output = |bs, seq_len, 2*hidden_size|
        output = torch.cat([output, x_emb], 2)
        # output = |bs, seq_len, embedding_dim + 2*hidden_size|
        output = self.tanh(self.W(output)).transpose(1, 2)
        # output = |bs, seq_len, hidden_size_linear| -> |bs, hidden_size_linear, seq_len|
        output = F.max_pool1d(output, output.size(2)).squeeze(2)
        # output = |bs, hidden_size_linear|
        output = self.fc(output)
        # output = |bs, class_num|
        return output

### 1. init
    def __init__(self, vocab_size, embedding_dim, hidden_size, hidden_size_linear, class_num, dropout):

In [44]:
embedding = nn.Embedding(vocab_size, embedding_dim, padding_idx=0) 
embedding
# https://wikidocs.net/64779
# vocab_size: 임베딩할 단어들의 개수, 단어 집합의 크기
# embedding_dim: 임베딩할 벡터의 차원, 사용자가 지정하는 하이퍼파라미터
# padding_idx: 패딩을 위한 토큰의 인덱스를 알려주는 선택적으로 사용하는 인자

Embedding(8000, 300, padding_idx=0)

In [45]:
embedding.weight # 단어 집합 크기의 행을 가지는 임베딩 테이블
# 그로 인해 모든 단어는 고유한 임베딩 벡터를 가질 수 있게 된다

Parameter containing:
tensor([[ 0.0000,  0.0000,  0.0000,  ...,  0.0000,  0.0000,  0.0000],
        [-0.0358,  0.7773,  0.1604,  ...,  1.2047, -0.1756,  0.6334],
        [-1.3885,  1.0440,  0.6876,  ..., -1.2369, -1.0200,  0.3204],
        ...,
        [ 0.5114, -0.6404, -0.7450,  ..., -0.2333, -0.4646, -0.7085],
        [ 0.0707, -0.9604,  0.4873,  ..., -0.4103, -0.2261,  0.3827],
        [ 1.4953, -0.3108,  0.3387,  ...,  0.0967,  0.7396,  0.9613]],
       requires_grad=True)

In [46]:
lstm = nn.LSTM(embedding_dim, hidden_size, batch_first=True, bidirectional=True, dropout=dropout)
lstm
# https://pytorch.org/docs/stable/generated/torch.nn.LSTM.html
# embedding_dim(input_size): Input 사이즈에 해당 하는 수
# hidden_size: 은닉층의 사이즈에 해당 하는 수
# batch_first: True일 시, Output 값의 사이즈는 (batch, seq, feature), 
# false이면(default), (seq, batch, feature)
# bidirectional: True일 시, 양방향 RNN

LSTM(300, 512, batch_first=True, bidirectional=True)

In [47]:
W = nn.Linear(embedding_dim + 2*hidden_size, hidden_size_linear)
W
# 파이토치의 선형회귀모델: nn.Linear(in_features,out_features)
# in_features: input_sample의 size, 300 + 2*512
# out_features: output_sample의 size

Linear(in_features=1324, out_features=512, bias=True)

In [48]:
tanh = nn.Tanh()
tanh

Tanh()

In [49]:
fc = nn.Linear(hidden_size_linear, class_num)
fc

Linear(in_features=512, out_features=4, bias=True)

In [50]:
vocab_size, embedding_dim, hidden_size, hidden_size_linear, class_num, dropout

(8000, 300, 512, 512, 4, 0.0)

In [51]:
embedding.to(args.device), lstm.to(args.device), W.to(args.device), tanh.to(args.device), fc.to(args.device)

(Embedding(8000, 300, padding_idx=0),
 LSTM(300, 512, batch_first=True, bidirectional=True),
 Linear(in_features=1324, out_features=512, bias=True),
 Tanh(),
 Linear(in_features=512, out_features=4, bias=True))

In [53]:
x.shape

torch.Size([64, 64])

In [54]:
x.to(args.device)

tensor([[2075,  955,  519,  ...,    0,    0,    0],
        [3750,   19,   71,  ...,    0,    0,    0],
        [ 725,  845,   17,  ...,    0,    0,    0],
        ...,
        [3442, 2886,   82,  ...,    0,    0,    0],
        [   7, 3130,    1,  ...,    0,    0,    0],
        [ 160,  443, 2825,  ...,    0,    0,    0]], device='cuda:1')

In [55]:
y

tensor([2, 0, 3, 0, 1, 2, 0, 0, 1, 3, 1, 2, 1, 3, 2, 0, 1, 2, 3, 2, 3, 0, 2, 0,
        1, 2, 2, 3, 2, 2, 3, 0, 2, 3, 3, 0, 2, 2, 3, 0, 0, 1, 1, 3, 1, 3, 1, 2,
        1, 0, 3, 0, 2, 1, 0, 1, 3, 3, 0, 0, 0, 3, 3, 1], device='cuda:1')

### 2. forward
def forward(self, x):

In [56]:
# x = |bs, seq_len|
x_emb = embedding(x) # 컴퓨터가 계산하기 쉽도록 64개의 단어 각각을 300차원으로 임베딩.
x_emb.shape

torch.Size([64, 64, 300])

In [57]:
x_emb

tensor([[[-1.2345e+00, -5.0303e-01, -7.6857e-01,  ..., -1.2427e+00,
          -1.3255e+00,  1.9905e-02],
         [-5.3455e-01, -2.7590e-01, -1.1622e+00,  ..., -9.2333e-01,
           9.4101e-01, -3.5804e-02],
         [-6.4196e-01,  1.0714e+00, -6.2158e-01,  ...,  7.9481e-01,
           1.1765e+00, -2.5283e-01],
         ...,
         [ 0.0000e+00,  0.0000e+00,  0.0000e+00,  ...,  0.0000e+00,
           0.0000e+00,  0.0000e+00],
         [ 0.0000e+00,  0.0000e+00,  0.0000e+00,  ...,  0.0000e+00,
           0.0000e+00,  0.0000e+00],
         [ 0.0000e+00,  0.0000e+00,  0.0000e+00,  ...,  0.0000e+00,
           0.0000e+00,  0.0000e+00]],

        [[-1.8986e-01, -3.2915e-01,  2.5847e-01,  ...,  2.4800e-01,
           2.8761e-01, -4.4554e-01],
         [ 9.6857e-01,  2.1162e+00,  2.9777e-01,  ..., -6.1507e-01,
          -1.1115e+00,  4.2129e-01],
         [-1.2841e+00, -6.5953e-01,  1.0542e+00,  ...,  3.7605e-01,
           6.7920e-01,  3.7369e-01],
         ...,
         [ 0.0000e+00,  0

In [ ]:
#x_emb.to(args.device)

#### lstm에서 batch_first(변화1)
- True이면 Output은 (batch, seq, feature), 
- false이면(default), (seq, batch, feature)
- 해당 코드의 경우 batch와 seq가 모두 64로 같아서 shape에는 차이X

#### lstm에서 bidirectional(변화2)
- (batch, seq, feature)
- True 이면 hiddensize*2
- False이면 hiddensize

In [59]:
# 기존 방법
lstm = nn.LSTM(embedding_dim, hidden_size, batch_first=True, bidirectional=True, dropout=dropout)
lstm
lstm.to(args.device)
# x_emb = |bs, seq_len, embedding_dim|
output, _ = lstm(x_emb) 
# (lstm): LSTM(300, 512, batch_first=True, bidirectional=True) 
output.shape

torch.Size([64, 64, 1024])

In [61]:
# 변화1. batch_first=False로 둔 경우, output에는 변화X
lstmF = nn.LSTM(embedding_dim, hidden_size, batch_first=False, bidirectional=True, dropout=dropout)
lstmF
lstmF.to(args.device)
output3, _ = lstmF(x_emb) 
output3.shape

torch.Size([64, 64, 1024])

In [62]:
# 변화2. bidirectional=False인 경우 , output이 변화
lstm2 = nn.LSTM(embedding_dim, hidden_size, batch_first=True, bidirectional=False, dropout=dropout)
lstm2.to(args.device)

output2, _ = lstm2(x_emb) 
output2.shape # 1024가 512로 바뀜

torch.Size([64, 64, 512])

### torch.cat
cat함수는 concatenate를 해주는 함수로 concatenate하고자 하는 차원을 증가시킨다 (차원의 수는 유지된다). concatenate하고자 하는 차원을 지정해주면 그 차원으로 두 tensor의 차원을 더한 값으로 차원이 변경된다.

    output.shape  -- torch.Size([64, 64, 1024])
    x_emb.shape   -- torch.Size([64, 64, 300])

### torch.cat 예제

In [66]:
a = torch.tensor([[1, 2, 3],
                  [4, 5, 6]])

b = torch.tensor([[7, 8, 9],
                  [10, 11, 12]])

In [67]:
torch.cat([a, b], dim = 0)

tensor([[ 1,  2,  3],
        [ 4,  5,  6],
        [ 7,  8,  9],
        [10, 11, 12]])

In [68]:
torch.cat([a, b], dim = 1)

tensor([[ 1,  2,  3,  7,  8,  9],
        [ 4,  5,  6, 10, 11, 12]])

In [72]:
# output = |bs, seq_len, 2*hidden_size|
output = torch.cat([output, x_emb], 2) # 1024 + 300, 
output.shape

torch.Size([64, 64, 1324])

In [73]:
W(output)
# W = nn.Linear(embedding_dim + 2*hidden_size, hidden_size_linear)

tensor([[[-2.2298e-01, -3.0989e-01,  6.4601e-02,  ..., -1.0887e-01,
           1.0118e-01,  1.2446e-01],
         [-2.0040e-01, -2.6127e-01, -8.3947e-03,  ...,  2.6482e-01,
          -2.2881e-01, -5.8112e-01],
         [-7.1501e-01,  6.4244e-02,  1.8631e-01,  ..., -7.8846e-02,
          -4.3733e-01,  3.1097e-02],
         ...,
         [-2.3517e-02, -4.0162e-02, -2.4087e-02,  ..., -6.5100e-03,
          -6.3396e-03,  6.4006e-05],
         [-2.3226e-02, -3.9944e-02, -2.4010e-02,  ..., -7.4408e-03,
          -8.2661e-03,  5.2050e-04],
         [-2.2807e-02, -3.9334e-02, -2.3613e-02,  ..., -9.5613e-03,
          -1.1742e-02,  1.0115e-03]],

        [[ 2.6039e-01, -5.0749e-02,  9.2746e-02,  ..., -5.7363e-02,
           5.1620e-02, -1.5809e-02],
         [ 4.4953e-02,  7.4514e-02,  2.9245e-01,  ...,  8.2706e-02,
          -2.3725e-01, -1.7499e-01],
         [ 2.3553e-01, -5.2676e-01, -2.3357e-01,  ...,  3.4419e-01,
          -3.6252e-02, -4.1872e-01],
         ...,
         [-2.3340e-02, -3

In [74]:
tanh(W(output))

tensor([[[-2.1935e-01, -3.0033e-01,  6.4511e-02,  ..., -1.0845e-01,
           1.0083e-01,  1.2382e-01],
         [-1.9776e-01, -2.5549e-01, -8.3945e-03,  ...,  2.5879e-01,
          -2.2490e-01, -5.2348e-01],
         [-6.1381e-01,  6.4156e-02,  1.8418e-01,  ..., -7.8683e-02,
          -4.1143e-01,  3.1087e-02],
         ...,
         [-2.3512e-02, -4.0140e-02, -2.4083e-02,  ..., -6.5099e-03,
          -6.3395e-03,  6.4006e-05],
         [-2.3222e-02, -3.9922e-02, -2.4005e-02,  ..., -7.4407e-03,
          -8.2659e-03,  5.2050e-04],
         [-2.2803e-02, -3.9313e-02, -2.3609e-02,  ..., -9.5611e-03,
          -1.1742e-02,  1.0115e-03]],

        [[ 2.5466e-01, -5.0705e-02,  9.2481e-02,  ..., -5.7300e-02,
           5.1574e-02, -1.5808e-02],
         [ 4.4923e-02,  7.4376e-02,  2.8438e-01,  ...,  8.2518e-02,
          -2.3290e-01, -1.7322e-01],
         [ 2.3127e-01, -4.8290e-01, -2.2941e-01,  ...,  3.3122e-01,
          -3.6237e-02, -3.9585e-01],
         ...,
         [-2.3336e-02, -3

In [75]:
tanh(W(output)).shape

torch.Size([64, 64, 512])

In [76]:
tanh(W(output)).transpose(1, 2)

tensor([[[-2.1935e-01, -1.9776e-01, -6.1381e-01,  ..., -2.3512e-02,
          -2.3222e-02, -2.2803e-02],
         [-3.0033e-01, -2.5549e-01,  6.4156e-02,  ..., -4.0140e-02,
          -3.9922e-02, -3.9313e-02],
         [ 6.4511e-02, -8.3945e-03,  1.8418e-01,  ..., -2.4083e-02,
          -2.4005e-02, -2.3609e-02],
         ...,
         [-1.0845e-01,  2.5879e-01, -7.8683e-02,  ..., -6.5099e-03,
          -7.4407e-03, -9.5611e-03],
         [ 1.0083e-01, -2.2490e-01, -4.1143e-01,  ..., -6.3395e-03,
          -8.2659e-03, -1.1742e-02],
         [ 1.2382e-01, -5.2348e-01,  3.1087e-02,  ...,  6.4006e-05,
           5.2050e-04,  1.0115e-03]],

        [[ 2.5466e-01,  4.4923e-02,  2.3127e-01,  ..., -2.3336e-02,
          -2.2697e-02, -2.2202e-02],
         [-5.0705e-02,  7.4376e-02, -4.8290e-01,  ..., -3.6261e-02,
          -3.7965e-02, -3.8491e-02],
         [ 9.2481e-02,  2.8438e-01, -2.2941e-01,  ..., -2.6498e-02,
          -2.5983e-02, -2.5085e-02],
         ...,
         [-5.7300e-02,  8

In [77]:
tanh(W(output)).transpose(1, 2).shape

torch.Size([64, 512, 64])

In [78]:
# output = |bs, seq_len, embedding_dim + 2*hidden_size|
output = tanh(W(output)).transpose(1, 2)
# (W): Linear(in_features=1324, out_features=512, bias=True)
output

tensor([[[-2.1935e-01, -1.9776e-01, -6.1381e-01,  ..., -2.3512e-02,
          -2.3222e-02, -2.2803e-02],
         [-3.0033e-01, -2.5549e-01,  6.4156e-02,  ..., -4.0140e-02,
          -3.9922e-02, -3.9313e-02],
         [ 6.4511e-02, -8.3945e-03,  1.8418e-01,  ..., -2.4083e-02,
          -2.4005e-02, -2.3609e-02],
         ...,
         [-1.0845e-01,  2.5879e-01, -7.8683e-02,  ..., -6.5099e-03,
          -7.4407e-03, -9.5611e-03],
         [ 1.0083e-01, -2.2490e-01, -4.1143e-01,  ..., -6.3395e-03,
          -8.2659e-03, -1.1742e-02],
         [ 1.2382e-01, -5.2348e-01,  3.1087e-02,  ...,  6.4006e-05,
           5.2050e-04,  1.0115e-03]],

        [[ 2.5466e-01,  4.4923e-02,  2.3127e-01,  ..., -2.3336e-02,
          -2.2697e-02, -2.2202e-02],
         [-5.0705e-02,  7.4376e-02, -4.8290e-01,  ..., -3.6261e-02,
          -3.7965e-02, -3.8491e-02],
         [ 9.2481e-02,  2.8438e-01, -2.2941e-01,  ..., -2.6498e-02,
          -2.5983e-02, -2.5085e-02],
         ...,
         [-5.7300e-02,  8

In [79]:
output.shape

torch.Size([64, 512, 64])

#### (논문 3. Methodology) 1.2 Text Representation learning
- CNN의 사용, max-pooling
* max-pooling을 사용한 이유: 풀링레이어를 통해 다양한 길이의 텍스트를 고정된 길이로 변환하여 전체 문장에서 정보 포착이 가능하도록 하고자 하였다, 그 중에서도 단어의 조합을 사용하는 average-pooling이 아닌, 가장 중요한 latent semantic factor를 찾는 것을 목적으로 두어 max-pooling을 썼다

In [80]:
output.size() # torch.Size([64, 512, 64])
output.size(2)

64

contextwindowsize RCNN에서 어떻게 설정?

    - RCNN에서 kernel_size는 윈도우 크기와 연관이 없음, 64로 설정한 것은 전체를 다 보겠다는 뜻인데
     - 해당 모델(RCNN)에서 max_pooling을 사용한 것은 다양한 길이의 텍스트를 fixed_length(여기서는 64)로 변환해 전체 문장에서 정보를 포착해 가장 중요한 단어를 찾기 위함이고
     - max함수를 이용해 element-wise-function(각 행렬의 원소끼리만 곱하는 것)을 시행하는 것이 목표임 --> 이 과정을 해당 논문에서 recurrent structure라고 부른다.
     - 결론적으로 굳이 따지자면 윈도우 숫자를 조정한 CNN과 달리 계속 64이다?

1. output | torch.Size([64, 512, 64])
2. output.size(2) | 64
-> 두 가지가 element-wise 되어 shape가 최종적으로 torch.Size([64, 512, 1])로 변환(가장 중요한 latent semantic factor가 document에서 찾아진 느낌)

In [81]:
F.max_pool1d(output, output.size(2)) #  output.size(2)로 설정했다는 것은 전체를 한번에 다 보겠다는 의미, 

# torch.nn.functional.max_pool1d과 같은 것
# https://pytorch.org/docs/stable/generated/torch.nn.functional.max_pool1d.html
# torch.nn.functional.max_pool1d(input, kernel_size, stride=None, padding=0, dilation=1, ceil_mode=False, return_indices=False)
# output: input tnesor의 shape
# output.size(2): kernel_size로 윈도우의 크기

tensor([[[0.3830],
         [0.4417],
         [0.2895],
         ...,
         [0.4633],
         [0.5335],
         [0.6317]],

        [[0.5340],
         [0.3255],
         [0.3734],
         ...,
         [0.4088],
         [0.4308],
         [0.5111]],

        [[0.3268],
         [0.4867],
         [0.4567],
         ...,
         [0.5977],
         [0.3470],
         [0.4843]],

        ...,

        [[0.3467],
         [0.5059],
         [0.3676],
         ...,
         [0.5677],
         [0.4602],
         [0.6396]],

        [[0.4683],
         [0.3742],
         [0.5175],
         ...,
         [0.5593],
         [0.4158],
         [0.2340]],

        [[0.3045],
         [0.5428],
         [0.4797],
         ...,
         [0.3339],
         [0.5254],
         [0.5762]]], device='cuda:1', grad_fn=<SqueezeBackward1>)

In [82]:
F.max_pool1d(output, output.size(2)).shape # 최종적으로 torch.Size([64, 512, 1])

torch.Size([64, 512, 1])

In [83]:
# output = |bs, seq_len, hidden_size_linear| -> |bs, hidden_size_linear, seq_len|
output = F.max_pool1d(output, output.size(2)).squeeze(2)
# torch.squeeze(input, dim=None, *, out=None) → Tensor
# https://pytorch.org/docs/stable/generated/torch.squeeze.html
output

tensor([[0.3830, 0.4417, 0.2895,  ..., 0.4633, 0.5335, 0.6317],
        [0.5340, 0.3255, 0.3734,  ..., 0.4088, 0.4308, 0.5111],
        [0.3268, 0.4867, 0.4567,  ..., 0.5977, 0.3470, 0.4843],
        ...,
        [0.3467, 0.5059, 0.3676,  ..., 0.5677, 0.4602, 0.6396],
        [0.4683, 0.3742, 0.5175,  ..., 0.5593, 0.4158, 0.2340],
        [0.3045, 0.5428, 0.4797,  ..., 0.3339, 0.5254, 0.5762]],
       device='cuda:1', grad_fn=<SqueezeBackward1>)

In [84]:
output.shape

torch.Size([64, 512])

In [85]:
# output = |bs, hidden_size_linear|
output = fc(output)
output
# fc = nn.Linear(hidden_size_linear, class_num)
# Linear(in_features=512, out_features=4, bias=True)

tensor([[-0.4005, -0.2630, -0.1167,  0.2234],
        [-0.4743, -0.2382, -0.1708,  0.1335],
        [-0.4905, -0.2309, -0.0922,  0.1613],
        [-0.5241, -0.2523, -0.1607,  0.1328],
        [-0.3450, -0.1673, -0.0877,  0.1468],
        [-0.4680, -0.2153, -0.2045,  0.0974],
        [-0.4088, -0.2255, -0.0799,  0.0932],
        [-0.3792, -0.2392, -0.0131,  0.1349],
        [-0.4910, -0.3037, -0.1469,  0.1715],
        [-0.4721, -0.3496, -0.1047,  0.0859],
        [-0.5137, -0.2236, -0.1837,  0.1721],
        [-0.4266, -0.2343, -0.1882,  0.1555],
        [-0.4333, -0.1742, -0.0794,  0.1783],
        [-0.4727, -0.1833, -0.0585,  0.1371],
        [-0.3569, -0.1839, -0.1016,  0.1540],
        [-0.3420, -0.2582, -0.1118,  0.1211],
        [-0.4790, -0.1723, -0.1282,  0.1286],
        [-0.5094, -0.2216, -0.2185,  0.2052],
        [-0.4209, -0.2806, -0.0876,  0.1456],
        [-0.4251, -0.1311, -0.0318,  0.1541],
        [-0.3290, -0.2635, -0.0926,  0.1306],
        [-0.4699, -0.2333, -0.1451

In [86]:
output.shape

torch.Size([64, 4])

# train

In [87]:
def train(model, optimizer, train_dataloader, valid_dataloader, args):
    best_f1 = 0
    for epoch in range(1, args.epochs+1): # 그냥 시행시 range(1,11)
        model.train()
        for step, (x, y) in enumerate(train_dataloader): 
            # step이 1719인 이유, 1719*64=110016임, 마지막 배치 사이즈가 48로 64보다 16작기 때문에, traindata의 수를 64로 나누고 1을 더한 값
            x, y = x.to(args.device), y.to(args.device)
            pred = model(x)
            loss = F.cross_entropy(pred, y)

            optimizer.zero_grad()
            loss.backward()
            optimizer.step()

        avg_loss, accuracy, _, _, f1, _ = evaluate(model, valid_dataloader, args)

        if f1 > best_f1:
            best_f1 = f1
            if not os.path.isdir(args.model_save_path):
                os.mkdir(args.model_save_path)
            torch.save(model.state_dict(), os.path.join(args.model_save_path, "best.pt"))

In [88]:
best_f1 = 0

In [89]:
model.train()

RCNN(
  (embedding): Embedding(8000, 300, padding_idx=0)
  (lstm): LSTM(300, 512, batch_first=True, bidirectional=True)
  (W): Linear(in_features=1324, out_features=512, bias=True)
  (tanh): Tanh()
  (fc): Linear(in_features=512, out_features=4, bias=True)
)

In [90]:
pred = model(x)
pred

tensor([[ 1.6784e-02,  5.7946e-01, -3.0091e-01,  1.5225e-01],
        [ 3.6258e-02,  6.1641e-01, -4.0627e-01,  1.4236e-01],
        [ 2.6025e-02,  5.9532e-01, -4.4350e-01,  2.0270e-01],
        [ 2.4278e-02,  6.4957e-01, -4.4139e-01,  9.9292e-02],
        [ 7.0093e-02,  6.2640e-01, -3.0061e-01,  1.8934e-01],
        [ 5.4416e-02,  5.6373e-01, -4.4139e-01,  1.8902e-01],
        [ 1.4685e-02,  4.9146e-01, -4.0513e-01,  2.0586e-01],
        [-3.5756e-02,  4.9156e-01, -3.4017e-01,  1.6545e-01],
        [ 5.2441e-02,  5.1138e-01, -3.3040e-01,  1.9392e-01],
        [-5.0310e-02,  4.9321e-01, -3.5128e-01,  1.6536e-01],
        [ 7.8132e-03,  5.6272e-01, -3.4274e-01,  1.8820e-01],
        [-3.3753e-02,  5.8660e-01, -4.0247e-01,  1.5094e-01],
        [ 3.5996e-02,  5.3973e-01, -3.9896e-01,  1.2463e-01],
        [ 3.3402e-02,  5.9913e-01, -4.9253e-01,  1.6979e-01],
        [ 7.0332e-02,  5.6449e-01, -2.7201e-01,  1.3308e-01],
        [ 5.9773e-02,  6.0540e-01, -3.3478e-01,  2.3437e-01],
        

### output 레이어 소프트맥스 함수 적용?

#### F.cross_entropy()
F.log_softmax() + F.nll_loss() = F.cross_entropy()

https://wikidocs.net/60572

* nll(Negative Log Likelihood): nll_loss는 F.log_softmax()를 수행한 후에 남은 수식들을 수행
* 결론적으로, F.cross_entropy는 비용 함수에 소프트맥스 함수까지 포함하고 있음
* (논문에서 output레이어에 소프트맥스 함수가 적용된다, 그리고 이것이 output numbers를 확률로 바꾼다~~ 하는 부분에 해당)

In [91]:
loss = F.cross_entropy(pred, y)
loss

tensor(1.4543, device='cuda:1', grad_fn=<NllLossBackward>)

### torch.optim 
* https://algopoolja.tistory.com/55
    * gradients값들을 추후에 backward를 해줄때 계속 더해주기 때문에 시행, 따라서 backpropagation을 하기전에 gradients를 zero로 만들어주고 시작해야 함
* torch.optim- 최적화 단계(신경망을 훈련시키는 동안, 네트워크의 가중치나 매개변수를 조정하기 위해 오차를 역전파하는 것), 모델을 훈련하는 동안 다양한 유형의 최적화 스케줄을 실행할 수 있도록 함.

In [92]:
optimizer.zero_grad() 
optimizer

Adam (
Parameter Group 0
    amsgrad: False
    betas: (0.9, 0.999)
    eps: 1e-08
    lr: 0.0003
    weight_decay: 0
)

In [93]:
loss.backward()
loss

tensor(1.4543, device='cuda:1', grad_fn=<NllLossBackward>)

In [94]:
optimizer.step()
optimizer

Adam (
Parameter Group 0
    amsgrad: False
    betas: (0.9, 0.999)
    eps: 1e-08
    lr: 0.0003
    weight_decay: 0
)

#### 1. epoch을 range(1,3)으로 수정한 경우 + evaluate
- 해당 코드의 경우 evaluate 함수에 따로 train 관련하여 수정하는 코드가 없다.
- 그렇기 떄문에 trainers에서 epoch을 수정한 후 이어서 evaluate를 바로 진행하면 그에 맞추어서 결과가 나온다

In [ ]:
best_f1 = 0
for epoch in range(1, 3): # 그냥 시행시 range(1,11)
    model.train()
    for step, (x, y) in enumerate(train_dataloader): 
        # step이 1719인 이유, 1719*64=110016임, 마지막 배치 사이즈가 48로 64보다 16작기 때문에, traindata의 수를 64로 나누고 1을 더한 값
        x, y = x.to(args.device), y.to(args.device)
        pred = model(x)
        loss = F.cross_entropy(pred, y)

        optimizer.zero_grad()
        loss.backward()
        optimizer.step()

    avg_loss, accuracy, _, _, f1, _ = evaluate(model, valid_dataloader, args)

    if f1 > best_f1:
        best_f1 = f1
        if not os.path.isdir(args.model_save_path):
            os.mkdir(args.model_save_path)
        torch.save(model.state_dict(), os.path.join(args.model_save_path, "best.pt"))

In [ ]:
evaluate(model, test_dataloader,\

#### 2. epoch을 수정하지 않고 훈련한 경우

In [ ]:
train(model, optimizer, train_dataloader, valid_dataloader, args)

# evaluate

In [ ]:
def evaluate(model, valid_dataloader, args):
    with torch.no_grad(): # 메모리 사용량을 아껴주는 gradient 연산에 쓰이는 파이썬 컨텍스트 매니저
        model.eval()
        losses, correct = 0, 0
        y_hats, targets = [], []
        for x, y in valid_dataloader:
            x, y = x.to(args.device), y.to(args.device)
            pred = model(x)
            loss = F.cross_entropy(pred, y)
            losses += loss.item()

            y_hat = torch.max(pred, 1)[1] # torch.max: 입력된 Tensor의 최대값을 반환
            y_hats += y_hat.tolist()
            targets += y.tolist()
            correct += (y_hat == y).sum().item()

    avg_loss, accuracy, precision, recall, f1, cm = metrics(valid_dataloader, losses, correct, y_hats, targets)
    return avg_loss, accuracy, precision, recall, f1, cm

In [ ]:
# avg_loss, accuracy, precision, recall, f1, cm의 경우 utils.py에서 함수로 정의

def metrics(dataloader, losses, correct, y_hats, targets):
    avg_loss = losses / len(dataloader)
    accuracy = correct / len(dataloader.dataset) * 100
    precision = precision_score(targets, y_hats, average='macro')
    recall = recall_score(targets, y_hats, average='macro')
    f1 = f1_score(targets, y_hats, average='macro')
    cm = confusion_matrix(targets, y_hats)
    return avg_loss, accuracy, precision, recall, f1, cm

# test set에 대한 결과

In [ ]:
from sklearn.metrics import precision_score, recall_score, f1_score, confusion_matrix

evaluate(model, test_dataloader, args) # accuracy, precision, recall, f1 순으로 결과 출력
# _, accuracy, precision, recall, f1, cm = evaluate(model, test_dataloader, args)

# cm=confusionmatrix: def evaluate에서 만든 y_hats, targets = [], [] 리스트 만들고, correct += (y_hat == y).sum().item() 사용해서 맞춘 개수 표현, cm = confusion_matrix(targets, y_hats)